In [1]:
import numpy as np
from scipy.sparse import *
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold 


Read dataset

In [2]:
dataset = pd.read_csv('netflixSorted.csv', sep=',')

dataset = dataset.drop(['Previous Movie', 'Date'],axis = 1)
YY = dataset['Rate'].values
XX = dataset.drop(['Rate'], axis=1)


OneHotEncoder

In [3]:
XX = OneHotEncoder(sparse=True).fit_transform(XX.values)

D:\anaconda\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [4]:
def sumOfDiff(A, B):
	u = 0
	sum = 0
	for i in A:
		sum = sum + (i - B[u])**2
		u = u + 1
	return sum

def getErr(Y, y_pred, N):
	sum_data2 = sumOfDiff(Y, y_pred)
	return np.sqrt(sum_data2 / N)
	#r2 =  1 - sum_data2/sum([data**2 for data in(Y - Y.mean())])

	#return float(rmse), float(r2)



Факторизационная машину 2-го порядка с квадратичной функцией потерь (аналогично линейной регрессии)


In [49]:
class factorization_machine:

    def get_batches(self, dataset, batch_size):
        X, Y = dataset
        n_samples = X.shape[0]

        # Shuffle at the start of epoch
        indices = np.arange(n_samples)
        np.random.shuffle(indices)

        for start in range(0, n_samples, batch_size):
            end = min(start + batch_size, n_samples)
            batch_idx = indices[start:end]

            yield X[batch_idx], Y[batch_idx]
            
    def prediction(self, X, W, V,X2,X_V, n):
        tmp = np.sum(X_V**2-(X2).dot(V**2), axis = 1, keepdims=True)
        return X.dot(W.reshape(n,1)) + np.multiply(1/2, tmp )
    
    def cost(self, curr, pred):
        return np.subtract(pred, curr.reshape(len(curr),1))

    def step(self, cost, x, n, W, learning_rate):
        return W.reshape(n,1) - np.multiply(learning_rate, (x.T).dot(cost))
    
    def fit(self, X, Y, n, kk):
        
        W = np.random.random(n)
        V = np.random.random((n, k))
        
        for i in range(epochs):
            print(i)
            
            for x_batch, y_batch in self.get_batches((X,Y), batch_size):
                learning_rate = kk /((np.sqrt(i + 1))*x_batch.shape[0])
                
                X2 = x_batch.power(2)
                X_V = x_batch.dot(V)
                
                y_pred = self.prediction(x_batch, W, V, X2, X_V, XX.shape[1])
                loss = self.cost(y_batch, y_pred)

                W = self.step(loss, x_batch, XX.shape[1] , W, learning_rate)
                loss = np.multiply(learning_rate, loss)
                
                for j in range(k):
                    gradV = x_batch.multiply(X_V[:, j].reshape(-1, 1)) - X2.multiply(V[:, j])
                    V[:,j] = V[:,j] - loss.reshape(-1) @ gradV
                    
        return W, V

Кроссвалидация

In [47]:
def cross_validation(kk):
    kf = KFold(n_splits=3) 
    kf.get_n_splits(XX) 
    
    data = pd.DataFrame(columns=["T1", "T2", "T3",  "E", "STD"])
    
    count = 0
    for train_i, test_i in kf.split(XX):
        print(count)
        print('begin')
        
        factorizationMachine = factorization_machine()
        
        X_train, X_test = XX[train_i], XX[test_i]
        Y_train, Y_test = YY[train_i], YY[test_i]
        
        n = XX.shape[1]
        
        W,V = factorizationMachine.fit(X_train, Y_train, n, kk)
                
        X2 = X_train.power(2)
        X_V = X_train.dot(V)
        y_pred = factorizationMachine.prediction(X_train, W, V, X2,X_V,n).reshape(-1)
        rmse_train = getErr(Y_train, y_pred, len(Y_train))
        print (rmse_train)
        
        X2 = X_test.power(2)
        X_V = X_test.dot(V)
        y_pred = factorizationMachine.prediction(X_test, W, V,X2, X_V,n).reshape(-1)
        rmse_test = getErr(Y_test, y_pred, len(Y_test))
        print (rmse_test)
        
        count = count + 1
        data["T" + str(count)] = [ rmse_test, rmse_train] 
        data["E"] = data[["T1", "T2", "T3"]].mean(axis=1)
        data["STD"] = data[["T1", "T2", "T3"]].std(axis=1)
        print('end')
        
        break
        
    data.index = ["RMSE_test", "RMSE_train"]     
    print(data)


In [50]:
k = 3
epochs = 5
batch_size = 1000

cross_validation(1.6)


0
begin
0
1
2
3
4
0.9653800545216137
1.0374054608372618
end
                  T1   T2   T3         E  STD
RMSE_test   1.037405  NaN  NaN  1.037405  NaN
RMSE_train  0.965380  NaN  NaN  0.965380  NaN


<module 'scipy.sparse.data' from 'C:\\Users\\ivm\\Anaconda3\\lib\\site-packages\\scipy\\sparse\\data.py'>
